# Introducción

El siguiente notebook contiene una implementación del problema de estudio, incluyendo las funciones más importantes, explicaciones y algunas pruebas con los datos de prueba.

En paralelo a esto, se realizó una API que realiza las mismas funciones que el notebook, pero utilizando SQL para ingenierir y almacenar datos bajo demanda y realizar las consultas de manera más eficiente. La API se encuentra dockerizada y se puede ejecutar con:
    
```bash
docker compose build
docker compose up
```

Luego, toda la documentación de los endpoints puede encontrarse en `localhost:8000/docs`.

Adicionalmente se implementó el módulo `loader.py` que permite cargar fácilmente los datos de prueba en la base de datos SQL. Para esto, se debe ejecutar el siguiente comando desde la carpeta raíz del repositorio:

```bash
python loader.py
```

Por defecto se cargarán los datos en `data/trips.csv`, pero se puede especificar cualquier otro archivo CSV si se ejecuta `python loader.py [filename]`.

# 1. Procesos automatizados para ingerir y almacenar los datos bajo demanda

In [2]:
import numpy as np
import pandas as pd

from datetime import datetime as dt, date
from scipy.spatial import cKDTree
from shapely import wkt
from shapely.geometry import Point, box
from utils.converter import meters2degrees

## a. Agrupación de viajes similares

La agrupación se realiza en términos de origen, destino y hora del día.

### - Supuestos

Partimos por definir algunos supuestos:
- 2 puntos en el mapa se consideran cercanos solo si están a una distancia menor a 5 km
- 2 viajes se consideran similares temporalmente si se encuentran en un rango de 2 horas

Estos supuestos pueden ser modificados en la siguiente celda de definición de constantes:

In [14]:
MAX_NEIGHBORS_DISTANCE = 5000   # in meters
MAX_TIME_DIFFERENCE = 120       # in minutes

Un último supuesto es que se asumirá que la Tierra es una esfera perfecta de radio 6371.009 kms, lo que nos permitirá calcular de forma aproximada la distancia entre dos puntos en el mapa tanto en metros como en grados.

En base a este último supuesto, podemos calcular `MAX_NEIGHBORS_DEGREES` en base a `MAX_NEIGHBORS_DISTANCE`:

In [15]:
MAX_NEIGHBORS_DEGREES = meters2degrees(MAX_NEIGHBORS_DISTANCE)
MAX_NEIGHBORS_DEGREES

0.04496601677464458

### - Propuestas de optimización

#### - Optimización 1: Utilizar un algoritmo de clustering

Una alternativa para mejorar el rendimiento de la agrupación de viajes similares es partir utilizando un algoritmo de clustering. Por ejemplo, se puede KMeans para agrupar puntos en base a su cercanía si consideramos 3 dimensiones:
    - Distancia entre puntos de origen
    - Distancia entre puntos de destino
    - Diferencia de horas entre viajes

Esto nos permitiría reducir el número de viajes a comparar, porque podríamos buscar candidatos a viajes compatidos solo dentro del cluster (no dentro de todo el dataset).

Sin embargo, es necesario precisar que en este caso particular la clusterización no es un reemplazante del proceso de búsqueda de viajes similares mostrado a continuación, dado que dentro de un cluster siguen pudiendo existir viajes que no cumplen los supuestos de cercanía mínima definidos anteriormente. La clusterización es solo un paso previo opcional que nos permite reducir el número de viajes a comparar.

#### - Optimización 2: Segmentar por ciudad

Finalmente, es necesario precisar que el dataset a utilizar incluye viajes en las ciudades de Praga, Turin y Hamburgo. Una alternativa que optimizaría la búsqueda de viajes similares sería trabajar con cada ciudad como un dataset separado.

Sin embargo para efectos de este estudio se utilizará el dataset completo, bajo la premisa de que si el modelo se ejecuta eficientemente en este caso, es más probable que escale bien al ser aplicado en una sola ciudad pero con más datos.

### - Carga de datos

Luego, importamos el dataset, transformamos las coordenadas en objetos de shapely y los tiempos en datetime:

In [16]:
TRIPS_DATASET = "data/trips.csv"

trips = pd.read_csv(TRIPS_DATASET)

trips['origin_coord']      = trips['origin_coord'].apply(wkt.loads)
trips['destination_coord'] = trips['destination_coord'].apply(wkt.loads)


# NOTE: We only care about the hours, not the dates. However, since objects of
# type datetime.time cannot be subtracted, the current date is added to all
# values to take advantage of the subtraction properties of datetime.datetime.
date_format="%Y-%m-%d %H:%M:%S"
trips['datetime'] = trips['datetime'].apply(
    lambda x: dt.combine(date.today(), dt.strptime(x, date_format).time())
)

Limpiamos las columnas que no vamos a utilizar:

_Dejaremos la columna `'region'` porque nos permitirá corroborar que no se están agrupando viajes de diferentes ciudades._

In [17]:
trips.drop(['datasource'], axis=1, inplace=True)
trips.head()

,region,origin_coord,destination_coord,datetime
0,Prague,POINT (14.4973794438195 50.00136875782316),POINT (14.43109483523328 50.04052930943246),2023-01-15 09:03:40
1,Turin,POINT (7.672837913286881 44.9957109242058),POINT (7.720368637535126 45.06782385393849),2023-01-15 02:54:04
2,Prague,POINT (14.32427345662177 50.00002074358429),POINT (14.47767895969969 50.09339790740321),2023-01-15 08:52:25
3,Turin,POINT (7.541509189114433 45.09160503827746),POINT (7.74528653441973 45.02628598341506),2023-01-15 09:49:16
4,Turin,POINT (7.614078119815749 45.13433106465422),POINT (7.527497142312585 45.03335051325654),2023-01-15 12:45:54


### - Búsqueda de viajes similares

Hacemos uso de KD-Tree y las operaciones matriciales de pandas para realizar búsquedas óptimas de viajes similares.

_Esta implementación de `ckdnearest` difiere ligeramente de la de la API, dado que las coordenadas de los puntos son guardadas de forma diferente en el DataFrame y en SQL._

In [18]:
# Based on https://gis.stackexchange.com/questions/222315/finding-nearest-point-in-other-geodataframe-using-geopandas
def ckdnearest(
    df: pd.DataFrame,
    column: str,
    distance: float=MAX_NEIGHBORS_DEGREES
):
    """Find points that are less than the indicated distance from each point in
    the indicated column.

    Argument
    ---------
    df: pd.DataFrame
        A pandas DataFrame (or GeoPandasDataframe) with at least one column of
        coordinates
    column: str
        The column of the df used for search the nearest points. Elements in
        this columns must be shapely.geometry.point.Point.
    distance: float (optional)
        The maximum distance a point can be to be considered a neighbor. The
        default value is MAX_NEIGHBORS_DEGREES.

    Returns
    -------
    df_nearest: pd.DataFrame
        A pandas DataFrame with the same columns as the input df, plus a column
        with the nearest points named {column}_neighbors.
    """
    coords = np.array(list(df[column].apply(lambda x: (x.x, x.y))))
    btree = cKDTree(coords)
    # Find the nearest points
    idx = btree.query_ball_tree(btree, r=distance)
    for i, neighbors in enumerate(idx):
        neighbors.remove(i)
    df_nearest = pd.concat(
        [
            df,
            pd.Series(idx, name=f'{column}_neighbors')
        ], 
        axis=1)
    return df_nearest


def cotemporals(df: pd.DataFrame, column: str, time: int=MAX_TIME_DIFFERENCE):
    """Find points that are less than the indicated time difference from each
    point in the indicated column.

    Argument
    ---------
    df: pd.DataFrame
        A pandas DataFrame with at least one column of datetimes.
    column: str
        The column of the df used for search the cotemporal trips. Elements in
        this columns must be datetime.datetime.
    time: int (optional)
        The maximum time difference a point can be to be considered a neighbor.
        The default value is MAX_TIME_DIFFERENCE.

    Returns
    -------
    df_nearest: pd.DataFrame
        A pandas DataFrame with the same columns as the input df, plus a column
        with the nearest points named {column}_neighbors.
    """
    delta = pd.Timedelta(time, unit='m')
    def get_neighbors(x):
        return df[
            (
                (
                    (df[column] >= x[column]) &
                    (df[column] <= x[column] + delta)
                ) | (
                    (df[column] <= x[column]) &
                    (df[column] >= x[column] - delta)
                )
            ) & (df.index != x.name)
        ].index.to_list()
    df_nearest = pd.concat(
        [
            df,
            pd.Series(
                df.apply(get_neighbors, axis=1),
                name=f'{column}_neighbors'
            )
        ],
        axis=1
    )
    return df_nearest

Buscamos los viajes similares en base al punto de origen y los guardamos en el DataFrame:

In [385]:
trips = ckdnearest(trips, 'origin_coord')
trips.head()

,region,origin_coord,destination_coord,datetime,origin_coord_neighbors
0,Prague,POINT (14.4973794438195 50.00136875782316),POINT (14.43109483523328 50.04052930943246),2023-01-15 09:03:40,"[15, 47]"
1,Turin,POINT (7.672837913286881 44.9957109242058),POINT (7.720368637535126 45.06782385393849),2023-01-15 02:54:04,"[43, 63, 65]"
2,Prague,POINT (14.32427345662177 50.00002074358429),POINT (14.47767895969969 50.09339790740321),2023-01-15 08:52:25,[64]
3,Turin,POINT (7.541509189114433 45.09160503827746),POINT (7.74528653441973 45.02628598341506),2023-01-15 09:49:16,"[16, 68, 74, 84, 87]"
4,Turin,POINT (7.614078119815749 45.13433106465422),POINT (7.527497142312585 45.03335051325654),2023-01-15 12:45:54,"[41, 58]"


Buscamos los viajes similares en base al punto de destino y los guardamos en el DataFrame:

In [386]:
trips = ckdnearest(trips, 'destination_coord')
trips.head()

,region,origin_coord,destination_coord,datetime,origin_coord_neighbors,destination_coord_neighbors
0,Prague,POINT (14.4973794438195 50.00136875782316),POINT (14.43109483523328 50.04052930943246),2023-01-15 09:03:40,"[15, 47]","[36, 67, 95]"
1,Turin,POINT (7.672837913286881 44.9957109242058),POINT (7.720368637535126 45.06782385393849),2023-01-15 02:54:04,"[43, 63, 65]","[18, 20, 31, 33, 37, 49, 58, 84, 86]"
2,Prague,POINT (14.32427345662177 50.00002074358429),POINT (14.47767895969969 50.09339790740321),2023-01-15 08:52:25,[64],"[13, 73, 77, 95]"
3,Turin,POINT (7.541509189114433 45.09160503827746),POINT (7.74528653441973 45.02628598341506),2023-01-15 09:49:16,"[16, 68, 74, 84, 87]","[16, 31, 37, 46, 63, 65]"
4,Turin,POINT (7.614078119815749 45.13433106465422),POINT (7.527497142312585 45.03335051325654),2023-01-15 12:45:54,"[41, 58]","[41, 56, 76]"


Buscamos los viajes similares en base a la hora y los guardamos en el DataFrame:

In [387]:
trips = cotemporals(trips, 'datetime')
trips.head()

,region,origin_coord,destination_coord,datetime,origin_coord_neighbors,destination_coord_neighbors,datetime_neighbors
0,Prague,POINT (14.4973794438195 50.00136875782316),POINT (14.43109483523328 50.04052930943246),2023-01-15 09:03:40,"[15, 47]","[36, 67, 95]","[2, 3, 5, 11, 25, 48, 60, 63, 65, 66, 71, 73, ..."
1,Turin,POINT (7.672837913286881 44.9957109242058),POINT (7.720368637535126 45.06782385393849),2023-01-15 02:54:04,"[43, 63, 65]","[18, 20, 31, 33, 37, 49, 58, 84, 86]","[8, 13, 26, 28, 36, 42, 44, 47, 49, 50, 51, 54..."
2,Prague,POINT (14.32427345662177 50.00002074358429),POINT (14.47767895969969 50.09339790740321),2023-01-15 08:52:25,[64],"[13, 73, 77, 95]","[0, 3, 5, 25, 48, 60, 63, 65, 66, 71, 73, 79, ..."
3,Turin,POINT (7.541509189114433 45.09160503827746),POINT (7.74528653441973 45.02628598341506),2023-01-15 09:49:16,"[16, 68, 74, 84, 87]","[16, 31, 37, 46, 63, 65]","[0, 2, 5, 11, 25, 39, 48, 60, 63, 65, 66, 73, ..."
4,Turin,POINT (7.614078119815749 45.13433106465422),POINT (7.527497142312585 45.03335051325654),2023-01-15 12:45:54,"[41, 58]","[41, 56, 76]","[6, 11, 15, 17, 20, 22, 24, 29, 31, 37, 39, 40..."


Finalmente, calculamos los viajes similares en base al cumplimiento de los 3 criterios anteriores:

In [557]:
def get_similar_trips(trip):
    """Get similar trips to the indicated trip. Similar trips are trips that
    have the same origin and destination coordinates and are cotemporals.
    """
    origin_neighbors = set(trip.origin_coord_neighbors)
    destination_neighbors = set(trip.destination_coord_neighbors)
    time_neighbors = set(trip.datetime_neighbors)
    similar_trips = origin_neighbors & destination_neighbors & time_neighbors
    frozen_similar_trips = frozenset(similar_trips.union({trip.name}))
    return frozen_similar_trips if similar_trips else np.nan


trips['similar_trips'] = trips.apply(get_similar_trips, axis=1)
trips.head()

,region,origin_coord,destination_coord,datetime,origin_coord_neighbors,destination_coord_neighbors,datetime_neighbors,similar_trips
0,Prague,POINT (14.4973794438195 50.00136875782316),POINT (14.43109483523328 50.04052930943246),2023-01-15 09:03:40,"[15, 47]","[36, 67, 95]","[2, 3, 5, 11, 25, 48, 60, 63, 65, 66, 71, 73, ...",NaN
1,Turin,POINT (7.672837913286881 44.9957109242058),POINT (7.720368637535126 45.06782385393849),2023-01-15 02:54:04,"[43, 63, 65]","[18, 20, 31, 33, 37, 49, 58, 84, 86]","[8, 13, 26, 28, 36, 42, 44, 47, 49, 50, 51, 54...",NaN
2,Prague,POINT (14.32427345662177 50.00002074358429),POINT (14.47767895969969 50.09339790740321),2023-01-15 08:52:25,[64],"[13, 73, 77, 95]","[0, 3, 5, 25, 48, 60, 63, 65, 66, 71, 73, 79, ...",NaN
3,Turin,POINT (7.541509189114433 45.09160503827746),POINT (7.74528653441973 45.02628598341506),2023-01-15 09:49:16,"[16, 68, 74, 84, 87]","[16, 31, 37, 46, 63, 65]","[0, 2, 5, 11, 25, 39, 48, 60, 63, 65, 66, 73, ...",NaN
4,Turin,POINT (7.614078119815749 45.13433106465422),POINT (7.527497142312585 45.03335051325654),2023-01-15 12:45:54,"[41, 58]","[41, 56, 76]","[6, 11, 15, 17, 20, 22, 24, 29, 31, 37, 39, 40...",NaN


Filtramos los viajes que no tienen viajes similares:

In [558]:
similar_trips = trips.dropna()
similar_trips

,region,origin_coord,destination_coord,datetime,origin_coord_neighbors,destination_coord_neighbors,datetime_neighbors,similar_trips
31,Turin,POINT (7.685899281076795 45.07566566332665),POINT (7.715597191909829 45.05465181263087),2023-01-15 13:42:34,"[8, 28, 33, 70, 71, 86]","[1, 3, 18, 20, 33, 37, 49, 58, 70, 84]","[4, 6, 14, 15, 17, 20, 22, 24, 29, 33, 34, 37,...","(33, 31)"
33,Turin,POINT (7.662608916626361 45.09442558983316),POINT (7.724289698249433 45.07378523979249),2023-01-15 15:11:01,"[12, 28, 30, 31, 70, 86]","[1, 18, 20, 31, 37, 49, 58, 84, 86]","[14, 18, 29, 31, 34, 41, 46, 74, 78, 83]","(33, 31)"


Y obtenemos los viajes similares que cumplen con los supuestos iniciales:

In [562]:
similar_trips.similar_trips.unique()

array([frozenset({33, 31})], dtype=object)

Concluyendo que, con los datos actuales de obtiene un solo grupo de viajes similares. Implicando que las personas que realizaron los viajes 31 y 33 podrían realizar un viaje compartido.

## 2. Funcionalidades

### a. Promedio semanal de la cantidad de viajes para un área definida

Cargamos nuevamente el dataset original para trabajar con los datos originales:

In [36]:
TRIPS_DATASET = "data/trips.csv"

trips = pd.read_csv(TRIPS_DATASET)

trips['origin_coord']      = trips['origin_coord'].apply(wkt.loads)
trips['destination_coord'] = trips['destination_coord'].apply(wkt.loads)

date_format="%Y-%m-%d %H:%M:%S"
trips['datetime'] = trips['datetime'].apply(
    lambda x:dt.strptime(x, date_format)
)

El área se define a través de un bounding box y el campo 'region' del dataset. Con la función `average_weekly_trips` podemos definir a libertad el bounding box y la región a analizar:

_Para la API se realizó una versión de esta función con una query SQL, la cual es llamada desde el endpoint `GET /average-weekly-trips` y puede encontrarse en `db/events.py/get_average_weekly_trips`._

In [55]:
def average_weekly_trips(df, bbox, region=None,):
    """Average the number of trips per week for a given region and bounding
    box. A trip is considered to be in the bounding box if its origin and
    destination are in the bounding box.

    Arguments
    ---------
    df: pd.DataFrame
        A pandas DataFrame with the trips data. Must have the columns 'region',
        'origin_cord', 'destination_coord', and 'datetime'.
    bbox: tuple
        A tuple with the bounding box coordinates. The format is (min_lon,
        min_lat, max_lon, max_lat).
    region: str (optional)
        The region to filter the trips. If None, all the trips are considered.

    Returns
    -------
    int
        The average number of trips per week.
    """
    if not region:
        region = df['region']
    filter_df = df[
        (df['region'] == region) &
        (df['origin_coord'].apply(lambda x: x.within(box(*bbox)))) &
        (df['destination_coord'].apply(lambda x: x.within(box(*bbox))))
    ]
    pd.options.mode.chained_assignment = None
    init_date = filter_df.datetime.min()
    end_date = filter_df.datetime.max()
    return filter_df.shape[0]/((end_date - init_date).days/7)

#### - Algunos ejemplos:

Promedio semanal de la cantidad de viajes para la ciudad de Hamburgo:

In [56]:
hamburg_bbox = (9.5, 53, 10.5, 54)
average_weekly_trips(trips, hamburg_bbox, 'Hamburg')

6.533333333333333

Promedio semanal de la cantidad de viajes para la ciudad de Hamburgo (usando solo el bounding box):

In [57]:
hamburg_bbox = (9.5, 53, 10.5, 54)
average_weekly_trips(trips, hamburg_bbox)

6.533333333333333

Promedio semanal de la cantidad de viajes para la ciudad de Praga:

In [58]:
prague_bbox = (14.2, 49.9, 15, 50.2)
average_weekly_trips(trips, prague_bbox, 'Prague')

8.5

Promedio semanal de la cantidad de viajes para la ciudad de Turin:

In [59]:
turin_bbox = (7.5, 44.5, 8, 45.5)
average_weekly_trips(trips, turin_bbox, 'Turin')

9.172413793103447

Promedio semanal de la cantidad de viajes para el sector norte de la ciudad de Turin:

In [60]:
turin_north_bbox = (7.5, 45, 8, 45.5)
average_weekly_trips(trips, turin_north_bbox, 'Turin')

6.275862068965517

Promedio semanal de la cantidad de viajes en todo el dataset:

In [61]:
world_bbox = (-180, -90, 180, 90)
average_weekly_trips(trips, world_bbox)

23.333333333333336

### b. Informar sobre el estado de la ingesta de datos

Una forma de abordar este problema en la API, es implementar el patrón de diseño Observer. El cual nos permite notificar a los observadores cuando se produce un cambio en el estado de la ingesta de datos.

Para que los usuarios puedan suscribirse y desuscribirse a los cambios de estado, bastaría con implementar endpoints `/subscribe` y `/unsubscribe` respectivamente.